## 続き

### コンビ名→芸人名→出身地etc を抽出
[芸歴.net](https://www.geireki.net/)を参照

In [3]:
import os

In [14]:
folder_name_raw = "./data/raw"
folder_name_tidy = "./data/tidy"
print(os.path.exists(folder_name_raw))
print(os.path.exists(folder_name_tidy))


True
True


In [19]:
# コンビ名一覧のルートURL
combi_url_root = "https://www.geireki.net/lists/member/combi?disband=1"

In [20]:
# 124ページに分かれているのでリストを作成
combi_urls = [combi_url_root]
for i in range(123):
    combi_urls.append(combi_url_root + f"&page={i+2}")
combi_urls

['https://www.geireki.net/lists/member/combi?disband=1',
 'https://www.geireki.net/lists/member/combi?disband=1&page=2',
 'https://www.geireki.net/lists/member/combi?disband=1&page=3',
 'https://www.geireki.net/lists/member/combi?disband=1&page=4',
 'https://www.geireki.net/lists/member/combi?disband=1&page=5',
 'https://www.geireki.net/lists/member/combi?disband=1&page=6',
 'https://www.geireki.net/lists/member/combi?disband=1&page=7',
 'https://www.geireki.net/lists/member/combi?disband=1&page=8',
 'https://www.geireki.net/lists/member/combi?disband=1&page=9',
 'https://www.geireki.net/lists/member/combi?disband=1&page=10',
 'https://www.geireki.net/lists/member/combi?disband=1&page=11',
 'https://www.geireki.net/lists/member/combi?disband=1&page=12',
 'https://www.geireki.net/lists/member/combi?disband=1&page=13',
 'https://www.geireki.net/lists/member/combi?disband=1&page=14',
 'https://www.geireki.net/lists/member/combi?disband=1&page=15',
 'https://www.geireki.net/lists/member/co

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [7]:
# User-Agent の設定
dummy_user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'


In [21]:
# コンビ詳細ページへのURLリストを作成

combi_page_urls = []

for combi_url in combi_urls:
    request = requests.get(combi_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    combi_pages = soup.select("#top > div > div > div > table > tbody > tr")
    for combi_page in combi_pages:
        combi_page_a = combi_page.select_one("td:nth-child(1) > a")
        url = combi_page_a.get("href")
        combi_name = combi_page_a.get_text(strip=True)
        combi_page_urls.append({"combi_name": combi_name, "page_url": url})
    time.sleep(1)

combi_page_df = pd.DataFrame(combi_page_urls)
combi_page_df

,combi_name,page_url
0,アームストロング,https://www.geireki.net/entertainers/1
1,アイスクリーム,https://www.geireki.net/entertainers/2
2,アイデンティティ,https://www.geireki.net/entertainers/3
3,アイナラット,https://www.geireki.net/entertainers/4
4,アイロンヘッド,https://www.geireki.net/entertainers/5
...,...,...
1846,セッツァー,https://www.geireki.net/entertainers/3186
1847,レモンコマドリ,https://www.geireki.net/entertainers/3188
1848,まぐろ兄弟,https://www.geireki.net/entertainers/3189
1849,幸せのとり,https://www.geireki.net/entertainers/3194


In [24]:
# script_all.csv に出現するコンビ名と照応

script_df = pd.read_csv(os.path.join(folder_name_tidy, "script_all.csv"))
script_df

,combi,neta,turn,speech
0,くらげ,物忘れ,0,ども！くらげです。お願いします
1,くらげ,物忘れ,1,お願いします
2,くらげ,物忘れ,2,すみません、ありがとうございます！頑張っていきましょうねー
3,くらげ,物忘れ,3,あのー、俺さ、この前初めてさ、31アイスに行ったのよ
4,くらげ,物忘れ,4,あ、31。でも31ってさ、店内カラフルだし女子高生とか多いからなんか、おじさんの俺達は入りづらいな
...,...,...,...,...
48840,紳助竜介,暴走族,276,「ふん」
48841,紳助竜介,暴走族,277,「かわいそうにあいつ死んでしまいよったー、ええ奴やったのにー、言うてんのはその日だけやで
48842,紳助竜介,暴走族,278,「その日だけ」
48843,紳助竜介,暴走族,279,「３日もしてみい友達の家行って、あいつこないだ頭割れてなお前、なんぼ気色悪かったお前ー


In [32]:
# 表記ゆれを統一
spelling_unification = {"NONSTYLE": "NON STYLE", "ノンスタイル": "NON STYLE", "Ａマッソ": "Aマッソ",\
                        "トムブラウン": "トム・ブラウン", "トレインディエンジェル": "トレンディエンジェル",\
                        "かまいまち": "かまいたち", "おぎはやぎ": "おぎやはぎ", "夢路いとし喜味こいし": "夢路いとし・喜味こいし",\
                        "学天即": "ガクテンソク", "流れ星": "流れ星☆", "海原千里万里": "海原千里・万里",\
                        "海原やすよともこ": "海原やすよ・ともこ", "中田ダイマルラケット": "中田ダイマル・ラケット",\
                        "紳助竜介": "島田紳助"}


In [36]:
# script_all.csv に出現するコンビ名と照応

page_combi_set = set(combi_page_df["combi_name"].unique())

script_combi_set = set(script_df["combi"].unique())
script_combi_set_unified = {spelling_unification.get(item, item) for item in script_combi_set}
script_combi_set_unified - page_combi_set

{'THIS IS パン', 'ぼる塾', 'エンペラー', '四千頭身', '島田紳助', '軍艦', '陣内智則'}

In [40]:
# ぼる塾、四千頭身、陣内智則、島田紳助を手動で入れる

additional_items = [{"combi_name": "ぼる塾", "page_url": "https://www.geireki.net/entertainers/2970"}, 
                    {"combi_name": "四千頭身", "page_url": "https://www.geireki.net/entertainers/1837"},
                    {"combi_name": "陣内智則", "page_url": "https://www.geireki.net/entertainers/2300"},
                    {"combi_name": "島田紳助", "page_url": "https://www.geireki.net/entertainers/2261"}]

combi_page_df_added = pd.concat([combi_page_df, pd.DataFrame(additional_items)]).reset_index(drop=True)
combi_page_df_added

,combi_name,page_url
0,アームストロング,https://www.geireki.net/entertainers/1
1,アイスクリーム,https://www.geireki.net/entertainers/2
2,アイデンティティ,https://www.geireki.net/entertainers/3
3,アイナラット,https://www.geireki.net/entertainers/4
4,アイロンヘッド,https://www.geireki.net/entertainers/5
...,...,...
1850,もしもし☆コールミーテレフォン,https://www.geireki.net/entertainers/3195
1851,ぼる塾,https://www.geireki.net/entertainers/2970
1852,四千頭身,https://www.geireki.net/entertainers/1837
1853,陣内智則,https://www.geireki.net/entertainers/2300


In [43]:
# THIS IS パン、エンペラー、軍艦は不明なので無視
script_combi_set_unified - set(combi_page_df_added["combi_name"].unique())

{'THIS IS パン', 'エンペラー', '軍艦'}

In [49]:
# 台本書き起こしに出てくるコンビだけ検索する
focus_page = combi_page_df_added[combi_page_df_added["combi_name"].isin(script_combi_set_unified)].copy().reset_index(drop=True)
focus_page

,combi_name,page_url
0,アイロンヘッド,https://www.geireki.net/entertainers/5
1,アインシュタイン,https://www.geireki.net/entertainers/6
2,赤もみじ,https://www.geireki.net/entertainers/18
3,アキナ,https://www.geireki.net/entertainers/23
4,アンタッチャブル,https://www.geireki.net/entertainers/75
...,...,...
136,エバース,https://www.geireki.net/entertainers/3070
137,ぼる塾,https://www.geireki.net/entertainers/2970
138,四千頭身,https://www.geireki.net/entertainers/1837
139,陣内智則,https://www.geireki.net/entertainers/2300


In [75]:
# コンビページから芸人ページURLを取得

performer_page_urls = []

for i, row in focus_page.iterrows():
    combi_name = row["combi_name"]
    print(combi_name)
    page_url = row["page_url"]
    request = requests.get(page_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    tbody = soup.select_one("#top > div > table")
    member_td = tbody.find("th", string="メンバー").find_next_sibling()
    members = member_td.select("tbody > tr")
    for member in members:
        performer = member.select_one("td:nth-child(1) > a")
        performer_url = performer.get("href")
        performer_name = performer.get_text(strip=True)
        performer_page_urls.append({"combi_name": combi_name, "combi_url": page_url, "performer_name": performer_name, "performer_url": performer_url})
    time.sleep(1)

performer_page_df = pd.DataFrame(performer_page_urls)
performer_page_df


アイロンヘッド
アインシュタイン
赤もみじ
アキナ
アンタッチャブル
EXIT
囲碁将棋
インディアンス
ウーマンラッシュアワー
ウエストランド
おいでやすこが
オードリー
おぎやはぎ
オジンオズボーン
オズワルド
オリエンタルラジオ
ガクテンソク
カベポスター
かまいたち
カミナリ
からし蓮根
祇園
きつね
ギャロップ
キュウ
麒麟
キングコング
銀シャリ
金属バット
くらげ
くりぃむしちゅー
コウテイ
コマンダンテ
さや香
さらば青春の光
三四郎
サンドウィッチマン
三拍子
シシガシラ
磁石
しずる
霜降り明星
ジャイアントジャイアン
ジャルジャル
真空ジェシカ
スーパーマラドーナ
スピードワゴン
スリムクラブ
セルライトスパ
ダイアン
ダイタク
タイムキーパー
タイムマシーン3号
ダイヤモンド
ダウンタウン
たくろう
ダブルアート
男性ブランコ
ダンビラムーチョ
チーモンチョーチュウ
千鳥
チュートリアル
ツートライブ
ツービート
ティーアップ
天竺鼠
東京ダイナマイト
東京ホテイソン
トータルテンボス
トット
トム・ブラウン
トレンディエンジェル
とろサーモン
ナイチンゲールダンス
ナイツ
中川家
中田ダイマル・ラケット
流れ星☆
なすなかにし
ななまがり
錦鯉
ニッポンの社長
ニューヨーク
NON STYLE
博多華丸・大吉
馬鹿よ貴方は
爆笑問題
鼻エンジン
ハライチ
ハリガネロック
パンクブーブー
B&B
フースーヤ
風藤松原
フットボールアワー
プラス・マイナス
ブラックマヨネーズ
ぺこぱ
ますだおかだ
マヂカルラブリー
ママタルト
マユリカ
マルセイユ
ミキ
見取り図
宮下草薙
ミルクボーイ
メンバー
モグライダー
もも
モンスターエンジン
ヤーレンズ
夢路いとし・喜味こいし
吉田たち
ランジャタイ
リニア
令和ロマン
ロングコートダディ
和牛
笑い飯
尼神インター
海原千里・万里
海原やすよ・ともこ
Aマッソ
TEAM BANANA
Dr.ハインリッヒ
にぼしいわし
紅しょうが
相席スタート
納言
南海キャンディーズ
メイプル超合金
ゆにばーす
ラランド
ヨネダ2000
ぎょうぶ
エバース
ぼる塾
四千頭身
陣内智則
島田紳助


,combi_name,combi_url,performer_name,performer_url
0,アイロンヘッド,https://www.geireki.net/entertainers/5,辻井亮平,https://www.geireki.net/perfomers/1308
1,アイロンヘッド,https://www.geireki.net/entertainers/5,ナポリ,https://www.geireki.net/perfomers/2401
2,アインシュタイン,https://www.geireki.net/entertainers/6,稲田直樹,https://www.geireki.net/perfomers/210
3,アインシュタイン,https://www.geireki.net/entertainers/6,河井ゆずる,https://www.geireki.net/perfomers/561
4,アキナ,https://www.geireki.net/entertainers/23,秋山賢太,https://www.geireki.net/perfomers/34
...,...,...,...,...
268,四千頭身,https://www.geireki.net/entertainers/1837,後藤拓実,https://www.geireki.net/perfomers/780
269,四千頭身,https://www.geireki.net/entertainers/1837,都築拓紀,https://www.geireki.net/perfomers/1318
270,四千頭身,https://www.geireki.net/entertainers/1837,石橋遼大,https://www.geireki.net/perfomers/172
271,陣内智則,https://www.geireki.net/entertainers/2300,陣内智則,https://www.geireki.net/perfomers/1041


In [76]:
# csvに保存
performer_page_df.to_csv(os.path.join(folder_name_raw, "performer_page.csv"), index=False)

In [ ]:
# 芸人ページから、誕生日・年齢・出身地・事務所・芸歴を取得


In [ ]:
# 関東・関西・その他に分類
def region_from_prefecture(prefecture):
    kansai = {"大阪府", "京都府", "兵庫県", "奈良県", "滋賀県", "和歌山県", "三重県", "福井県"}
    kanto = {"東京都", "神奈川県", "埼玉県", "千葉県", "茨城県", "栃木県", "群馬県"}
    
    if prefecture in kansai:
        return "関西"
    elif prefecture in kanto:
        return "関東"
    else:
        return "その他"

余裕があったら追加で全ユニット情報から各芸人情報を抽出して、誕生日・年齢・出身地・事務所・芸歴を比較したい

https://www.geireki.net/lists/member